In [2]:
from twilio.rest import Client

def send_sms(animal_class, confidence):
    account_sid = "ACdceeb5336fea1a0e46fb663fc57df38a"
    auth_token = "00e4d2e355fbdee88cea71a6ba79ee88"
    client = Client(account_sid, auth_token)

    message_body = f"⚠ Alert: {animal_class} detected with {confidence:.2f} confidence!"

    message = client.messages.create(
        body=message_body,
        from_='+918233404139',
        to='+917014723440'
    )

    print("SMS sent:", message.sid)

In [ ]:
import cv2
import requests
import numpy as np
from PIL import Image
from io import BytesIO
import torch
from torchvision import transforms

# Load the PyTorch model
model = load_state_dict(torch.load("model_ram.pt"))
model.eval()

# Define class labels
class_names = ['Cow', 'Deer', 'Dog', 'Horse', 'Goat', 'Sheep', 'Lion', 'Tiger', 'Elephent', 'Hippo']

# Preprocessing (adjust if your model needs different input size or normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Match model input
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Example normalization
])

# ESP32-CAM capture URL
url = "http://192.168.1.91/capture"

while True:
    try:
        # Get image from ESP32-CAM
        response = requests.get(url, timeout=5)
        image = Image.open(BytesIO(response.content)).convert("RGB")

        # Preprocess image
        input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

        # Run inference
        with torch.no_grad():
            outputs = model(input_tensor)
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
            confidence, class_idx = torch.max(probabilities, dim=0)

        detected_class = class_names[class_idx]

        print(f"Detected: {detected_class} (Confidence: {confidence:.2f})")

        if confidence > 0.8:
            print(f"⚠ Alert: {detected_class} detected!")
            send_sms(detected_class, confidence)

    except Exception as e:
        print("Error:", e)
